# BPNN training and testing 

In [26]:
import torch
import os
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import argparse
from sklearn.model_selection import KFold
from torch.utils.data import DataLoader
import random
import pickle
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
import torch.nn as nn
import torch.nn.functional as F
from skimage import io,transform

import torch.optim as optim
import torchvision
from torchvision import datasets,transforms 

from sklearn.model_selection import train_test_split
import Model
from trainer import Trainer
from sklearn import preprocessing
from sklearn.model_selection import RandomizedSearchCV
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

In [38]:
def normalization(csv_file,mode,indices):
    Data = pd.read_csv(csv_file)
    if mode == "standardization":
        scaler = preprocessing.StandardScaler()
    elif mode == "minmax":
        scaler = preprocessing.MinMaxScaler()
    scaler.fit(Data.iloc[indices,1:])
    return scaler


class Datasets(torch.utils.data.Dataset):
    def __init__(self):
        self.image_dir = "./data/ROI_trab"
        self.labels = pd.read_csv("./Label_5p.csv")
    def __len__(self):
        return len(self.labels)
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        img_name = os.path.join(self.image_dir, str(self.labels.iloc[idx,0]))
        image = io.imread(img_name) # Loading Image
        image = image / 255.0 # Normalizing [0;1]
        image = image.astype('float32') # Converting images to float32
        labels = self.labels.iloc[idx,1] # Takes all corresponding labels
        #sample = {'image': image, 'label': labels}
        #if self.transform:
            #sample = self.transform(sample)
        X=image
        Y=labels
        return X,Y

In [4]:
class NNet(nn.Module):
    def __init__(self):
        super().__init__(n1,n2,n3)
        self.fc1 = nn.Linear(512*512,n1)
        self.fc2 = nn.Linear(n1,n2)
        self.fc3 = nn.Linear(n2,n3)
        #self.fc5 = nn.Linear(n3,20)
        self.fc4 = nn.Linear(n3,1)
    def forward(self,X):
        X = torch.flatten(X)
        X = F.relu(self.fc1(X))
        X = F.relu(self.fc2(X))
        X = F.relu(self.fc3(X))
        #x = F.relu(self.fc5(x))
        X = self.fc4(X)
        return X

In [32]:
def NNet(n1,n2,n3):
    super().__init__(n1,n2,n3)
    self.fc1 = nn.Linear(512*512,n1)
    self.fc2 = nn.Linear(n1,n2)
    self.fc3 = nn.Linear(n2,n3)
    #self.fc5 = nn.Linear(n3,20)
    self.fc4 = nn.Linear(n3,1)
    

Starting by creating options for training, testing and usage.

In [5]:
class Args:
    label_dir = "./Label_5p.csv"
    image_dir = "./data/ROI_trab"
    train_cross = "./cross_output.pkl"
    batch_size = 8
    model = "ConvNet" 
    nof = 8
    lr= 0.001
    nb_epochs = 5
    checkpoint_path= "./"
    mode= "Train"
    cross_val = False
    k_fold= 5
    n1= 240
    n2= 120
    n3 = 60
    nb_workers = 0
    norm_method="standardization"

opt = Args()
NB_DATA = 3991
NB_LABEL = 5
PERCENTAGE_TEST = 20
RESIZE_IMAGE = 512

Let's check if gpu is available 

In [6]:
if torch.cuda.is_available():  
  device = "cuda:0"
  print("running on gpu")
else:  
  device = "cpu"
  print("running on cpu")

running on cpu


We start by importing the Network. Here we choose *ConvNet* but you can also choose *ResNet50*, *ResNet101* and *Unet*. 

In [23]:
model = Model.ConvNet(features =opt.nof,out_channels=NB_LABEL,k1 = 3,k2 = 3,k3= 3).to(device)

In [33]:
model = NNet()


TypeError: NNet() missing 3 required positional arguments: 'n1', 'n2', and 'n3'

In [34]:
model = KerasClassifier(build_fn=NNet)

We prepare the splitting of the dataset, and we normalize parameters label.

In [8]:
index = range(NB_DATA)
split = train_test_split(index,test_size = 0.2,random_state=1)

In [9]:
print(split[0])

[3247, 3729, 2707, 2425, 3473, 902, 1448, 990, 3108, 1865, 2698, 1373, 1380, 1176, 1379, 305, 2374, 439, 713, 3333, 953, 1836, 1271, 3045, 2164, 2930, 58, 1475, 664, 1710, 1962, 650, 2978, 462, 2765, 50, 772, 2098, 1078, 306, 1759, 2424, 2241, 65, 1661, 3233, 870, 2289, 3767, 195, 3226, 3970, 1167, 2997, 3119, 3900, 1744, 3369, 2045, 1847, 1625, 2000, 2206, 2259, 2697, 396, 1595, 1856, 3719, 2606, 1776, 586, 3803, 957, 1006, 2299, 473, 2733, 3482, 3483, 3937, 2039, 2238, 2833, 2814, 1459, 3246, 407, 137, 797, 1140, 1417, 3420, 3403, 1047, 1903, 1398, 2682, 834, 516, 3071, 955, 2950, 220, 2998, 3261, 3596, 3070, 2183, 2608, 1221, 3279, 1649, 725, 1503, 1642, 282, 3466, 1918, 1065, 3366, 1404, 2907, 2213, 3096, 471, 1038, 1501, 1173, 1162, 11, 3598, 415, 988, 1906, 2135, 1546, 1179, 2117, 757, 2749, 2719, 1807, 544, 3173, 2520, 3661, 2847, 2323, 1248, 3311, 1782, 671, 945, 223, 613, 3257, 3683, 1338, 2469, 2688, 2726, 2328, 180, 3856, 1030, 125, 2142, 730, 616, 812, 2856, 2935, 111, 3065

In [ ]:
scaler = dataloader.normalization(opt.label_dir,opt.norm_method,split[0])

Dataset creation from label and images

In [17]:
datasets = dataloader.Datasets(csv_file = opt.label_dir, image_dir = opt.image_dir, opt=opt, indices =split[0]) # Create dataset
trainloader = DataLoader(datasets, batch_size = opt.batch_size,  num_workers = opt.nb_workers )

NameError: name 'dataloader' is not defined

Let's try RandomizedSearch here.

In [39]:
X,Y = Datasets()


ValueError: too many values to unpack (expected 2)

In [13]:
n1 = [random.randint(50, 150) for x in range(10)]
n2 = [random.randint(50, 150) for x in range(10)]
n3 = [random.randint(50, 150) for x in range(10)]
param_dist = dict(n1 = n1,
                  n2 = n2,
                  n3 = n3
                 )

In [14]:
n_iter_search = 16

In [35]:
random_search = RandomizedSearchCV(estimator=model,param_distributions=param_dist,n_iter=n_iter_search,n_jobs=-1,cv=5)

In [36]:
random_search.fit(datasets)

TypeError: Singleton array array(<module 'torchvision.datasets' from '/linkhome/rech/genlhc01/uki75tv/.local/lib/python3.7/site-packages/torchvision/datasets/__init__.py'>,
      dtype=object) cannot be considered a valid collection.

Here, let's create our cross validation dataset

RuntimeError: mat1 and mat2 shapes cannot be multiplied (1x33554432 and 262144x100)

In [ ]:
print(datasets)

RuntimeError: mat1 and mat2 shapes cannot be multiplied (1x33554432 and 262144x10)

In [ ]:
print(train_ds)